# Analysis of bbox3 and Strategy Pivot

The `bbox3` binary is the core optimization engine used by the top public kernel. However, it fails to run in our environment due to a GLIBC version mismatch (`GLIBC_2.34` required, but we have `GLIBC_2.31`).

This notebook analyzes the situation and prepares a pivot strategy. We cannot easily upgrade GLIBC in this restricted environment. We must either:
1. Find the source code for `bbox3` and compile it.
2. Implement a Python-based replacement (Simulated Annealing).
3. Find a compatible binary.

The web search results suggest that `bbox3` is indeed a custom C++ optimizer likely implementing Simulated Annealing or a similar heuristic for packing. One search result points to a GitHub repo `choltz95/simulated_annealing_pcb` as a potential reference for irregular polygon packing. Another result mentions that top teams are using "Simulated Annealing combined with Lattice Generation".

Since we cannot run the binary, and the source is not immediately available in the `research/datasets` (only the binary is there), we must implement our own optimizer.

This notebook will:
1. Verify the `bbox3` failure (already done, but good to document).
2. Prototype a Python-based Simulated Annealing optimizer for a single N (e.g., N=5) to verify we can improve on the baseline.
3. If successful, this Python implementation will replace `bbox3`.


In [ ]:
import subprocess
import os

# Verify bbox3 failure
bbox3_path = "research/datasets/bbox3"
if os.path.exists(bbox3_path):
    try:
        subprocess.run(["chmod", "+x", bbox3_path], check=True)
        result = subprocess.run([f"./{bbox3_path}", "--help"], capture_output=True, text=True)
        print("bbox3 output:", result.stdout)
        print("bbox3 error:", result.stderr)
    except Exception as e:
        print(f"bbox3 execution failed: {e}")
else:
    print("bbox3 binary not found.")


In [ ]:
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from decimal import Decimal, getcontext
import math
import random

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w  = Decimal("0.4")
        top_w  = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def get_score(trees, n):
    if not trees: return 0
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return (side ** 2) / n

def check_overlap(trees):
    # Simple O(N^2) overlap check for prototype
    polys = [t.polygon for t in trees]
    for i in range(len(polys)):
        for j in range(i + 1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                return True
    return False

# Simulated Annealing Prototype
def simulated_annealing(n, steps=1000):
    # Initialize random trees
    trees = []
    for _ in range(n):
        trees.append(ChristmasTree(
            center_x=random.uniform(-n/2, n/2),
            center_y=random.uniform(-n/2, n/2),
            angle=random.uniform(0, 360)
        ))
    
    current_score = get_score(trees, n)
    best_trees = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in trees]
    best_score = current_score
    
    temp = 1.0
    cooling_rate = 0.995
    
    print(f"Start SA for N={n}: Initial Score = {current_score:.6f}")
    
    for i in range(steps):
        # Perturb one tree
        idx = random.randint(0, n-1)
        original_tree = trees[idx]
        
        # Create new tree with perturbation
        new_x = float(original_tree.center_x) + random.gauss(0, temp)
        new_y = float(original_tree.center_y) + random.gauss(0, temp)
        new_angle = float(original_tree.angle) + random.gauss(0, temp * 10)
        
        trees[idx] = ChristmasTree(new_x, new_y, new_angle)
        
        # Check validity (hard constraint)
        if check_overlap(trees):
            # Revert if invalid
            trees[idx] = original_tree
            continue
            
        new_score = get_score(trees, n)
        delta = new_score - current_score
        
        # Metropolis criterion
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_score = new_score
            if current_score < best_score:
                best_score = current_score
                best_trees = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in trees]
                # print(f"Step {i}: New Best Score = {best_score:.6f}")
        else:
            # Revert
            trees[idx] = original_tree
            
        temp *= cooling_rate
        
    print(f"End SA for N={n}: Best Score = {best_score:.6f}")
    return best_score

# Run a quick test for N=5
simulated_annealing(5, steps=500)
